# 1. Set Up DF

In [1]:
import pandas as pd
import numpy as np
import re
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# 2. Pre-processing DFs for profiles & game data

In [2]:
df_data = pd.read_csv('playerData.csv').drop('Unnamed: 0',axis=1)
df_data = df_data[["teamID","year","#","name"]]
split = df_data['name'].str.split(',',n=2,expand=True)
df_data['first_middle'] = split[0].str.replace(' ','-').str.strip()
df_data['last'] = split[1].str.strip()
df_data = df_data.drop('name',axis=1)
df_data = df_data.drop_duplicates(["teamID","year","first_middle","last"])
print('Number of players: ' + str(df_data.shape[0]))
print('Missing data summary:')
print(df_data.isna().sum())
df_data.head(15)

Number of players: 247
Missing data summary:
teamID          0
year            0
#               0
first_middle    0
last            0
dtype: int64


,teamID,year,#,first_middle,last
0,CTC,2017,3,Le,Nguycn
1,CTC,2017,4,Thanh,Hucnh
2,CTC,2017,6,Nguycn,Hoang
3,CTC,2017,7,Du-Minh,An
4,CTC,2017,8,Ngcc,Nguycn
5,CTC,2017,11,Thanh,Dinh
6,CTC,2017,12,Hucnh,Nguycn
7,CTC,2017,13,Le-Van,Day
10,CTC,2017,21,Hamilton,DeAngelo
11,CTC,2017,23,Dinh-Duy,Tan


In [3]:
df_profile = pd.read_csv('player_profile.csv').drop('Unnamed: 0',axis=1).drop_duplicates()
df_profile['first_middle'] = df_profile['first_middle'].str.strip()
df_profile['last'] = df_profile['last'].str.strip()
df_profile.drop_duplicates(["teamID","year","first_middle","last"])
print('Number of players: ' + str(df_profile.shape[0]))
print('Missing data summary:')
print(df_profile.isna().sum())
df_profile.head()

Number of players: 251
Missing data summary:
teamID            0
year              0
#                 0
first_middle      0
last              0
pos1             41
pos2            217
nat               0
height_cm        94
dtype: int64


,teamID,year,#,first_middle,last,pos1,pos2,nat,height_cm
0,CTC,2017,21,Hamilton,DeAngelo,F,C,USA,203.0
1,CTC,2017,23,Dinh-Duy,Tan,G,F,Vietnam,192.0
2,CTC,2017,11,Thanh,Dinh,NaN,NaN,Vietnam,NaN
3,CTC,2017,8,Ngcc,Nguycn,NaN,NaN,Vietnam,NaN
4,CTC,2017,65,Huynh-Huu,Thang,F,NaN,Vietnam,NaN


# 3. Make changes to "#" column of Profile DF
Merge 2 dfs on names, and compare between the "#"-columns of both dfs, returning the values that exists. Right Outer Join preserves data information on the Data DF, which is more essential for later application; however, accuracies are not guaranteed.

In [7]:
df_profile = pd.merge(df_data,df_profile,on=["teamID","year","first_middle","last"],how='right')
df_profile['#_x'] = df_profile['#_x'].fillna(-1)
b = np.where(df_profile["#_x"]>=0,df_profile["#_x"],df_profile["#_y"])
df_profile["#"] = b.astype(int)
df_profile = df_profile.drop(columns=['#_x','#_y'])
df_profile.head()

,teamID,year,first_middle,last,pos1,pos2,nat,height_cm,#
0,CTC,2017,Le,Nguycn,C,NaN,Vietnam,NaN,3
1,CTC,2017,Thanh,Hucnh,NaN,NaN,Vietnam,NaN,4
2,CTC,2017,Nguycn,Hoang,NaN,NaN,Vietnam,NaN,6
3,CTC,2017,Du-Minh,An,G,F,Vietnam,180.0,7
4,CTC,2017,Ngcc,Nguycn,NaN,NaN,Vietnam,NaN,8


# 4. Test Inner Merge on ('teamID', 'year', '#') as a key for both DFs

In [11]:
merged = pd.merge(df_data,df_profile,on=['teamID','year','#'])
merged.head()

,teamID,year,#,first_middle_x,last_x,first_middle_y,last_y,pos1,pos2,nat,height_cm
0,CTC,2017,3,Le,Nguycn,Le,Nguycn,C,NaN,Vietnam,NaN
1,CTC,2017,4,Thanh,Hucnh,Thanh,Hucnh,NaN,NaN,Vietnam,NaN
2,CTC,2017,6,Nguycn,Hoang,Nguycn,Hoang,NaN,NaN,Vietnam,NaN
3,CTC,2017,7,Du-Minh,An,Du-Minh,An,G,F,Vietnam,180.0
4,CTC,2017,8,Ngcc,Nguycn,Ngcc,Nguycn,NaN,NaN,Vietnam,NaN


In [17]:
print("Size of Data DF (rows): "+str(df_data.shape[0]))
print("Size of Profile DF (rows): "+str(df_profile.shape[0]))
print("Size of Merged DF (rows): "+str(merged.shape[0]))
print("-------------------")
print("Data Loss on Merge (rows): "+str(df_data.shape[0]-merged.shape[0]))

Size of Data DF (rows): 247
Size of Profile DF (rows): 251
Size of Merged DF (rows): 227
-------------------
Data Loss on Merge (rows): 20


# 5. Update player_profile.csv

In [18]:
with open('player_profile.csv', 'w', encoding='utf-8') as f:
            df_profile.to_csv(f, header=f.tell()==0)